In [1]:
import os
import numpy as np
import pandas as pd
import nltk
import keras
import six
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,AveragePooling1D,Flatten,concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import *

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package punkt to /home/roberto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/roberto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roberto/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/roberto/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
#Abro el archivo en el que se encuentra el dataset de los problemas
with open('singleop.json', 'r') as f:
    datastore = json.load(f)

In [4]:
# Archivo que contiene un listado de nombres
nombres = pd.read_csv('nombres-2015.csv')
names = pd.read_csv('yob2019.txt', header=None)    

In [5]:
dataset2 = pd.read_csv('problemas_adicionales2.csv')

In [6]:
preguntas = []
respuestas = []
ecuaciones = []
alineacion = []

for item in datastore:
    preguntas.append(item['sQuestion'])
    respuestas.append(item['lSolutions'])
    ecuaciones.append(item['lEquations'])
    alineacion.append(item['lAlignments'])

In [7]:
# Necesito convertir el dataset en un problema de clasificacion para que la red neuronal pueda identificar
# si estoy tratando de resolver un problema de sumas, restas, multiplicaciones o divisiones.
# Esto va a crear una lista con el tipo de operacion y que va a ser el resultado a inferir.
operaciones = []
sumas =0
restas =0
multiplicaciones =0
divisiones = 0
otras = 0
#Clasifico las operaciones en 0 para sumas, 1 para restas, 2 para multiplicaciones, 3 para divisiones y 4 sino lo encuentro.
for operacion in ecuaciones:
    if (operacion[0].find('+')>=0):
        operaciones.append(0)
        sumas = sumas + 1
    elif (operacion[0].find('-') >= 0 ):
        operaciones.append(1)
        restas = restas + 1
    elif(operacion[0].find('*') >=0):
        operaciones.append(2)
        multiplicaciones = multiplicaciones + 1
    elif(operacion[0].find('/')):
        operaciones.append(3)
        divisiones = divisiones + 1
    else:
        operaciones.append(4)
        otras = otras + 1

print('Tengo ', sumas, ' sumas ', restas, ' restas, ', multiplicaciones, ' multiplicaciones, ', divisiones, ' divisiones y otras operaciones ', otras)

Tengo  159  sumas  162  restas,  117  multiplicaciones,  124  divisiones y otras operaciones  0


In [8]:
preguntas2 = dataset2['Preguntas'].tolist()
respuestas2 = dataset2['respuestas'].tolist()

In [9]:
preguntas3 = preguntas + preguntas2
respuestas3 = operaciones + respuestas2

In [10]:
#El listado de nombres lo voy a truncar a los 15K primeros, dado que el resto son nombres muy residuales.
nombres_ = nombres['nombre'][:15000]

In [11]:
names_= names[0]

In [12]:
nombres_ = nombres_.append(names_)

In [13]:
st_words = list(stop_words)

In [14]:
nom = nombres['nombre'].values.tolist()

In [15]:
nomb =  nom + st_words

In [16]:
nomb[-1]

'otros'

In [17]:
# El vector preguntas_sin, consiste en las preguntas a las que voy a eliminar todos los nombres propios que no
# anaden ningun valor al conjunto de preguntas. No quiero que esos nombres se procesen y por tanto los elimino.
def eliminar_palabras(dataset, stopw):
    preguntas_sin = []
    for palabras in dataset:
        frases = [word for word in palabras.split(' ') if word not in stopw]
        frases = " ".join(frases)
        preguntas_sin.append(frases)
    return preguntas_sin

In [18]:
preguntas_sin = eliminar_palabras(preguntas3, nomb)

In [19]:
len(preguntas_sin)

974

In [20]:
from sklearn.utils import shuffle

preguntas3, respuestas3 = shuffle(preguntas_sin,respuestas3)

In [21]:
preguntas3[1]

'La Sra. Hilt compró cucurucho helado 99 centavos. ¿Cuánto costarían 2 conos helado?'

In [22]:
respuestas3[1]

2

In [23]:
preguntas_w = []
tokenizer = RegexpTokenizer(r'\w+')

for i,pregunta in enumerate(preguntas_sin):
    palabras = tokenizer.tokenize(pregunta)
    preguntas_w.append(palabras)

In [24]:
preguntas_w


[['Si',
  '7',
  'tapas',
  'botella',
  'caja',
  'pone',
  '7',
  'tapas',
  'dentro',
  'cuántas',
  'tapas',
  'caja'],
 ['José',
  'comienza',
  '7',
  'tapas',
  'botella',
  'Recibe',
  'dos',
  '2',
  'Rebecca',
  'Con',
  'cuántas',
  'tapas',
  'botella',
  'termina',
  'José'],
 ['Bridget',
  '4',
  'chicles',
  '4',
  'chicles',
  'Si',
  'da',
  'chicles',
  'Bridget',
  'cuántos',
  'chicles',
  'Bridget'],
 ['empieza',
  '7',
  'caramelos',
  'compra',
  '8',
  'más',
  'Con',
  'cuántos',
  'caramelos',
  'termina',
  'Brenda'],
 ['Hay',
  '4',
  'cartas',
  'Se',
  'añaden',
  '3',
  'cartas',
  'más',
  'Cuántas',
  'total'],
 ['Hay',
  '4',
  'canicas',
  'Se',
  'añaden',
  '7',
  'canicas',
  'más',
  'Cuántas',
  'total'],
 ['Hay',
  '8',
  'naranjas',
  'Se',
  'añaden',
  '8',
  'naranjas',
  'más',
  'Cuántas',
  'total'],
 ['recoge',
  '2',
  'cacahuetes',
  'El',
  'padre',
  'da',
  '5',
  'más',
  'Cuántos',
  'cacahuetes',
  'Carol'],
 ['Hay',
  '9',
  'ca

In [25]:
import gensim
import gensim.downloader as api
from gensim.models import KeyedVectors

In [26]:
glove = gensim.models.KeyedVectors.load_word2vec_format('SBW-vectors-300-min5.txt')

In [27]:
word2index = {token: token_index for token_index, token in enumerate(glove.index2word)}
wordEmbeddings = [glove[key] for key in word2index.keys()]

In [28]:
palabras

['85',
 'céntimos',
 'quiere',
 'comprar',
 'postales',
 'Cada',
 'postal',
 'cuesta',
 '9',
 'céntimos',
 'Cuántas',
 'postales',
 'puede',
 'comprar']

In [29]:
def embed(preguntas):
    embedded = []
    maximo = 0
    for pregunta in preguntas:
        a_idx = []
        for palabra in pregunta:
            if palabra in word2index.keys():
                a_idx.append(word2index[palabra])
            elif palabra.lower() in word2index.keys():
                a_idx.append(word2index[palabra.lower()])
            elif palabra.isdigit():
                a_idx.append(100000 + int(palabra))
                if((100000 + int(palabra)) > maximo):
                    maximo = 100000 + int(palabra)
            else:
                a_idx.append('100000')
        embedded.append(a_idx)
    return embedded, maximo

def encontrar_maximo(preguntas):
    maxlen = 0
    tot = 0
    for pregunta in preguntas:
        if len(pregunta)>maxlen:
            maxlen = len(pregunta)
        tot = tot + len(pregunta)
    avg = tot / len(preguntas)
    return avg, maxlen

def padding(preguntas, maxm):
    padded = pad_sequences(preguntas, maxlen = maxm, padding = padding_type, truncating = trunc_type)
    return padded

In [30]:
avg, maxlen = encontrar_maximo(preguntas_sin)

In [31]:
# Calcular el tamano del vocabulario
vocabulario = []

for pregunta in preguntas_w:
    for palabra in pregunta:
        vocabulario.append(palabra)

In [32]:
vocabulario = set(vocabulario)

In [33]:
len(vocabulario)

2305

In [34]:
# El tamano del vocabulario es necesario para poder crear la matriz de embeddings, con el numero de palabras
# totales que tengo
embedding_dim = 16
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [35]:
print("La media de las preguntas es de ", avg, " y la longitud máxima de la cadena es de ", maxlen)

La media de las preguntas es de  82.62731006160165  y la longitud máxima de la cadena es de  219


In [36]:
wordEmbeddings, vocab_size = embed(preguntas_sin)

In [37]:
vocab_size = vocab_size + 1

In [38]:
preguntas_sin[100]

'Heather comienza 86 bloques. comparte 41 Jose. ¿Con cuantos bloques termina Heather?'

In [39]:
len(wordEmbeddings[0])

65

In [40]:
padded = padding(wordEmbeddings, maxlen)

In [41]:
len(padded)

974

In [42]:
training_X = np.asarray(padded[:800])
test_X = np.asarray(padded[800:])
training_y = np.asarray(respuestas3[:800])
test_y = np.asarray(respuestas3[800:])


In [43]:
training_X

array([[   450,   1184, 100000, ...,      0,      0,      0],
       [  2251,     27,    657, ...,      0,      0,      0],
       [   680,   9383,   1184, ...,      0,      0,      0],
       ...,
       [  3736,    624, 100000, ...,      0,      0,      0],
       [  1378,     54,    894, ...,      0,      0,      0],
       [   727,    624, 100000, ...,      0,      0,      0]], dtype=int32)

In [44]:
"""
for t in training_y:
    if(type(s)!=np.int32):
        print(type(s))
"""

'\nfor t in training_y:\n    if(type(s)!=np.int32):\n        print(type(s))\n'

In [45]:
training_X[300]

array([100002, 100000,   3494,    993,    574,      7,    624,     54,
          657, 100000,    657,     54,    624,   3494,      7,    518,
           27,    657, 100000,   9383,      7,    892,      7, 100000,
         2649,   9383,    401,     27,   2079, 100000, 100000,   3736,
          624, 100000,   3494,    993,    574,   2649,    624, 100000,
          993,    624,     54, 100000,     54,   2079,   2079,     27,
          657, 100000, 100000, 100000,    271,    993,   2649,    624,
         3494,     27,    657, 100000,   3494,    993,    574,      7,
          624,     54,    657, 100000,   3494,     27,   3494,      7,
         2079, 100000,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [46]:
"""
for t in test_X:
    if(len(t)!=219):
        print(t)
"""

'\nfor t in test_X:\n    if(len(t)!=219):\n        print(t)\n'

In [47]:
# Creamos un modelo donde el primer argumento de la capa embedding son las palabras totales que voy a procesar
# vectorizadas en un indice.
# El segundo argumento, es el tamano del vector embedding, que he fijado en 16.
# El tercer argumento, es el tamano o longitud maxima que he definido para las preguntas. Numero total de palabras
# por pregunta.
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=maxlen),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(36, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [48]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 219, 16)           1600160   
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 36)                612       
_________________________________________________________________
dropout (Dropout)            (None, 36)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               4736      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 5

In [49]:
history = model.fit(training_X, training_y, epochs = 300, validation_data=[test_X, test_y], batch_size = 128)

Epoch 1/300
7/7 [==============================] - 0s 32ms/step - loss: 1.3835 - accuracy: 0.3288 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/300
7/7 [==============================] - 0s 19ms/step - loss: 1.3751 - accuracy: 0.3650 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/300
7/7 [==============================] - 0s 19ms/step - loss: 1.3653 - accuracy: 0.3650 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/300
7/7 [==============================] - 0s 17ms/step - loss: 1.3558 - accuracy: 0.3650 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/300
7/7 [==============================] - 0s 22ms/step - loss: 1.3482 - accuracy: 0.3650 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/300
7/7 [==============================] - 0s 17ms/step - loss: 1.3436 - accuracy: 0.3650 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/300
7/7 [==============================] - 0s 16ms/step - loss: 1.3397 - accuracy: 0.3650 - val_loss: 0.

7/7 [==============================] - 0s 16ms/step - loss: 1.3297 - accuracy: 0.3837 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 57/300
7/7 [==============================] - 0s 17ms/step - loss: 1.3283 - accuracy: 0.3738 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 58/300
7/7 [==============================] - 0s 18ms/step - loss: 1.3291 - accuracy: 0.3713 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 59/300
7/7 [==============================] - 0s 18ms/step - loss: 1.3298 - accuracy: 0.3725 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 60/300
7/7 [==============================] - 0s 16ms/step - loss: 1.3296 - accuracy: 0.3738 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 61/300
7/7 [==============================] - 0s 16ms/step - loss: 1.3275 - accuracy: 0.3775 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 62/300
7/7 [==============================] - 0s 17ms/step - loss: 1.3236 - accuracy: 0.3738 - val_loss: 0.0000e+

Epoch 111/300
7/7 [==============================] - 0s 19ms/step - loss: 1.2979 - accuracy: 0.4162 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 112/300
7/7 [==============================] - 0s 20ms/step - loss: 1.2959 - accuracy: 0.4050 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 113/300
7/7 [==============================] - 0s 18ms/step - loss: 1.2922 - accuracy: 0.4137 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 114/300
7/7 [==============================] - 0s 17ms/step - loss: 1.2932 - accuracy: 0.4137 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 115/300
7/7 [==============================] - 0s 18ms/step - loss: 1.2891 - accuracy: 0.4100 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 116/300
7/7 [==============================] - 0s 18ms/step - loss: 1.2940 - accuracy: 0.4150 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 117/300
7/7 [==============================] - 0s 19ms/step - loss: 1.2894 - accuracy: 0.4250 

7/7 [==============================] - 0s 18ms/step - loss: 1.2602 - accuracy: 0.4350 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 166/300
7/7 [==============================] - 0s 20ms/step - loss: 1.2662 - accuracy: 0.4325 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 167/300
7/7 [==============================] - 0s 19ms/step - loss: 1.2600 - accuracy: 0.4450 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 168/300
7/7 [==============================] - 0s 17ms/step - loss: 1.2593 - accuracy: 0.4350 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 169/300
7/7 [==============================] - 0s 17ms/step - loss: 1.2619 - accuracy: 0.4387 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 170/300
7/7 [==============================] - 0s 16ms/step - loss: 1.2604 - accuracy: 0.4425 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 171/300
7/7 [==============================] - 0s 18ms/step - loss: 1.2549 - accuracy: 0.4425 - val_loss: 0.

Epoch 220/300
7/7 [==============================] - 0s 18ms/step - loss: 1.2462 - accuracy: 0.4487 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 221/300
7/7 [==============================] - 0s 17ms/step - loss: 1.2446 - accuracy: 0.4437 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 222/300
7/7 [==============================] - 0s 16ms/step - loss: 1.2422 - accuracy: 0.4550 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 223/300
7/7 [==============================] - 0s 16ms/step - loss: 1.2469 - accuracy: 0.4450 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 224/300
7/7 [==============================] - 0s 16ms/step - loss: 1.2508 - accuracy: 0.4350 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 225/300
7/7 [==============================] - 0s 16ms/step - loss: 1.2398 - accuracy: 0.4437 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 226/300
7/7 [==============================] - 0s 17ms/step - loss: 1.2425 - accuracy: 0.4450 

7/7 [==============================] - 0s 17ms/step - loss: 1.2442 - accuracy: 0.4500 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 275/300
7/7 [==============================] - 0s 16ms/step - loss: 1.2407 - accuracy: 0.4338 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 276/300
7/7 [==============================] - 0s 16ms/step - loss: 1.2473 - accuracy: 0.4525 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 277/300
7/7 [==============================] - 0s 17ms/step - loss: 1.2371 - accuracy: 0.4450 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 278/300
7/7 [==============================] - 0s 18ms/step - loss: 1.2308 - accuracy: 0.4588 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 279/300
7/7 [==============================] - 0s 16ms/step - loss: 1.2306 - accuracy: 0.4563 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 280/300
7/7 [==============================] - 0s 17ms/step - loss: 1.2252 - accuracy: 0.4712 - val_loss: 0.

In [50]:
model2 = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences= True)),
    keras.layers.LSTM (64, dropout = 0.2, recurrent_dropout = 0.2, return_sequences = True),
    keras.layers.LSTM (128, dropout = 0.2, recurrent_dropout = 0.2, return_sequences = True),
    #keras.layers.LSTM (256, dropout = 0.2, recurrent_dropout = 0.2, return_sequences = True),
    keras.layers.LSTM (64, dropout = 0.2, recurrent_dropout = 0.2),
    keras.layers.Dense(5, activation='softmax')
])
opt = keras.optimizers.Adam(learning_rate = 0.01)
model2.compile(loss='sparse_categorical_crossentropy',optimizer=opt, metrics=['accuracy'])

In [51]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 219, 16)           1600160   
_________________________________________________________________
bidirectional (Bidirectional (None, 219, 256)          148480    
_________________________________________________________________
lstm_1 (LSTM)                (None, 219, 64)           82176     
_________________________________________________________________
lstm_2 (LSTM)                (None, 219, 128)          98816     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
Total params: 1,979,365
Trainable params: 1,979,365
Non-trainable params: 0
____________________________________________

In [ ]:
history2 = model2.fit(training_X, training_y, epochs = 240, validation_data=[test_X, test_y], batch_size= 16)

Epoch 1/240
